In [1]:
#import tensorflow as tf
import keras
from keras import backend as k
#from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils

import matplotlib.pyplot as plt
import numpy as np
#from tensorflow.keras.datasets import mnist
from sklearn.decomposition import PCA
from data_pretreat import handle_data
from PIL import Image
import cv2

k.set_image_data_format('channels_last')

# import keras.backend.tensorflow_backend as KTF
# KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'gpu':0})))

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
train_x,train_y,test_x,test_y=handle_data(train_scale=0.8)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

num_classes=16

(8194, 200)
(8194, 1)
(2055, 200)
(2055, 1)


In [3]:
#降维
def pca(x,n=196):
    pca=PCA(n_components=n)
    new_train_x=pca.fit_transform(x)
    return new_train_x

In [4]:
#pca+nor+reshape+add channel
# train_x=train_x/np.max(train_x)
# train_x=pca(train_x)
# train_x=train_x.reshape(train_x.shape[0],14,14)
# train_x=np.expand_dims(train_x,axis=3)

# test_x=test_x/np.max(test_x)
# test_x=pca(test_x)
# test_x=test_x.reshape(test_x.shape[0],14,14)
# test_x=np.expand_dims(test_x,axis=3)

data_all=np.r_[train_x,test_x]
#归一化
data_all=(data_all-np.min(data_all))/(np.max(data_all)-np.min(data_all))
#双边滤波

data_all=pca(data_all)
train_x=data_all[0:train_x.shape[0],:]
test_x=data_all[train_x.shape[0]:,:]
print(train_x.shape)
print(test_x.shape)
train_x=train_x.reshape(train_x.shape[0],14,14)
train_x=np.expand_dims(train_x,axis=3)
test_x=test_x.reshape(test_x.shape[0],14,14)
test_x=np.expand_dims(test_x,axis=3)

(8194, 196)
(2055, 196)


In [5]:
#one-hot
train_y=np_utils.to_categorical(train_y)[:,1:17]
test_y =np_utils.to_categorical(test_y)[:,1:17]
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(8194, 14, 14, 1)
(8194, 16)
(2055, 14, 14, 1)
(2055, 16)


In [6]:
#建模
def s_minst_cnn_model(input_shape,num_classes=16):
    model = Sequential()
    #模型1
    model.add(Conv2D(128, kernel_size=(1, 1), activation='relu',input_shape=input_shape))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    
    #模型2
#     model.add(Conv2D(16, kernel_size=(3, 3),
#                      activation='relu',
#                      input_shape=input_shape))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(32, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     #model.add(Dropout(0.2))
#     model.add(Flatten())
#     model.add(Dense(64, activation='relu'))
#     #model.add(Dropout(0.2))
#     model.add(Dense(num_classes, activation='softmax'))
#     model.summary()
    
    #模型3
#     model.add(Conv2D(32, kernel_size=(5, 5),
#                      activation='relu',
#                      input_shape=input_shape))
#     #model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
#     model.add(Conv2D(32, (3, 3), activation='relu'))
#     model.add(Flatten())
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(num_classes, activation='softmax'))
#     model.summary()

    return model;


In [7]:
s_model=s_minst_cnn_model(train_x[0].shape)
s_model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  #optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

#模型1
s_model.fit(train_x, train_y,
            batch_size=32,
            epochs=100,
            verbose=1)

#模型2
# s_model.fit(train_x, train_y,
#             batch_size=16,
#             epochs=70,
#             verbose=1)

#模型3
# s_model.fit(train_x, train_y,
#             batch_size=32,
#             epochs=140,
#             verbose=1)
    
print("Saving model to disk \n")
path="s_cnn_model.h5"
s_model.save(path)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 12, 12, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)  

8194/8194 [==============================] - 1s 151us/step - loss: 0.3746 - acc: 0.8782
Epoch 56/100
8194/8194 [==============================] - 1s 154us/step - loss: 0.3592 - acc: 0.8860
Epoch 57/100
8194/8194 [==============================] - 1s 152us/step - loss: 0.3635 - acc: 0.8856
Epoch 58/100
8194/8194 [==============================] - 1s 150us/step - loss: 0.3514 - acc: 0.8849
Epoch 59/100
8194/8194 [==============================] - 1s 153us/step - loss: 0.3602 - acc: 0.8822
Epoch 60/100
8194/8194 [==============================] - 1s 153us/step - loss: 0.3471 - acc: 0.8854
Epoch 61/100
8194/8194 [==============================] - 1s 154us/step - loss: 0.3547 - acc: 0.8911
Epoch 62/100
8194/8194 [==============================] - 1s 152us/step - loss: 0.3498 - acc: 0.8887
Epoch 63/100
8194/8194 [==============================] - 1s 154us/step - loss: 0.3382 - acc: 0.8926
Epoch 64/100
8194/8194 [==============================] - 1s 153us/step - loss: 0.3409 - acc: 0.8927
Epo

In [8]:
print(test_y.shape)
score = s_model.evaluate(test_x, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(2055, 16)
Test loss: 1.1023579119038502
Test accuracy: 0.8238442822384429
